In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from astropy.table import Table
import gravity_obs.p2_tools as p2t

In [3]:
table_path = '../../gvprep/tabs/star_k9-12mag_grp10-12mag_gaia_pairs.ipac'
tb = Table.read(table_path, format='ascii.ipac')

In [4]:
tb

name,ra_2000_hms,dec_2000_dms,ra_2000,dec_2000,parallax,pm,pmra,pmdec,radial_velocity,Grp,G,ra_2016,dec_2016,source_id,name_ft,ra_2000_hms_ft,dec_2000_dms_ft,ra_2000_ft,dec_2000_ft,H_ft,K_ft,parallax_ft,pm_ft,pmra_ft,pmdec_ft,radial_velocity_ft,Grp_ft,G_ft,ra_2016_ft,dec_2016_ft,source_id_ft,sobj_x,sobj_y,sep,pa
,,,,,mas,mas / yr,mas / yr,mas / yr,km / s,mag,mag,deg,deg,,,,,,,mag,mag,mas,mas / yr,mas / yr,mas / yr,km / s,mag,mag,deg,deg,,mas,mas,arcsec,deg
str28,str12,str13,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,str28,str12,str13,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64
Gaia DR3 2341844906852746368,00:00:02.943,-19:50:48.367,0.012260791705641448,-19.846768705463113,3.5114,29.492,0.426,-29.489,-10.1,10.817062,11.769627,0.01226280585,-19.84689976729,2341844906852746368,Gaia DR3 2341844906852744960,00:00:02.880,-19:50:47.733,0.012000535043,-19.84659248,9.7840003967,9.7069997787,4.0482,28.909,0.724,-28.899,-6.23,10.772411,12.115517,0.01200395497,-19.846720922,2341844906852744960,873.5902877630645,-649.6303633691983,1.0886595426877728,126.63570122745283
Gaia DR3 2314519534641619072,00:00:48.419,-31:24:08.551,0.2017465628668493,-31.402375276095544,1.9224,76.271,40.103,-64.878,--,11.25193,12.474266,0.20195538217,-31.40266362203,2314519534641619072,Gaia DR3 2314519534642211968,00:00:48.458,-31:24:08.366,0.20191001276,-31.402323892,10.355999947,10.262000084,1.8048,74.96,38.581,-64.269,--,11.243553,12.437418,0.20211090932,-31.402609531,2314519534642211968,-462.9589566323816,-200.70097727240034,0.5045908023381322,246.56240651398505
Gaia DR3 2414405355661710336,00:00:52.555,-17:38:52.103,0.2189781406721571,-17.64780632852596,2.6526,12.156,4.646,-11.234,-5.05,10.520113,11.390596,0.21899980915,-17.64785625575,2414405355661710336,Gaia DR3 2414405355662647808,00:00:52.477,-17:38:52.594,0.21865548352,-17.647942807,9.6160001755,9.5419998169,2.6362,12.297,5.671,-10.911,-6.41,10.340529,11.161269,0.2186819343,-17.6479913,2414405355662647808,1080.440578197648,482.99047282048576,1.183482843074383,65.91387453484865
Gaia DR3 2314335224710182528,00:02:31.762,-32:03:29.921,0.6323409784329383,-32.05831151437864,2.9534,16.078,5.455,-15.124,--,10.797275,11.574282,0.63236958442,-32.05837873164,2314335224710182528,Gaia DR3 2314335229005732992,00:02:31.748,-32:03:30.687,0.63228524427,-32.058524173,10.130000114,10.067000389,1.8595,15.252,4.579,-14.548,--,10.776859,12.335009,0.6323092588,-32.058588832,2314335229005732992,192.64661820872644,750.711930192581,0.7750362066643917,14.392592943043438
Gaia DR3 2421635331809030016,00:03:00.360,-11:58:27.625,0.7514999241450625,-11.974340378172274,1.6997,21.328,2.86,-21.135,18.69,10.828333,11.583986,0.75151291712,-11.97443431253,2421635331809030016,Gaia DR3 2421635331810404608,00:03:00.293,-11:58:27.847,0.75122255594,-11.974401856,10.081000328,9.9790000916,1.4545,20.799,2.181,-20.684,20.57,10.775371,12.19432,0.75123246451,-11.974493786,2421635331810404608,994.3194374037781,209.68065717669384,1.0161875425299292,78.09201742865505
Gaia DR3 2415871829296257280,00:03:04.311,-15:29:37.615,0.7679619005080659,-15.493781986570562,4.7633,41.227,-35.507,-20.951,17.62,12.13384,12.959554,0.76779813945,-15.49387510144,2415871829296257280,Gaia DR3 2415871829295257984,00:03:04.223,-15:29:38.465,0.76759580927,-15.494018133,10.151000023,10.010000229,5.2841,41.318,-36.465,-19.428,17.61,11.998395,13.019423,0.76742762979,-15.49410448,2415871829295257984,1294.7578033451086,810.8243532448048,1.527689072140633,57.94370718519913
Gaia DR3 2313395936838051968,00:05:41.769,-32:47:07.833,1.4240354265988613,-32.785509054880166,5.9683,24.931,-22.28,-11.187,8.6,11.967402,13.564322,1.42391764093,-32.7855587765,2313395936838051968,Gaia DR3 2313395936837386112,00:05:41.809,-32:47:08.384,1.4242029899,-32.785662123,10.286999702,10.098999977,5.997,27.227,-23.048,-14.496,1.59,11.962945,13.573459,1.424081147

In [10]:
tb['K'] = tb['G']-2.5
tb['H'] = tb['G']-2.2

In [5]:
tb.add_index('name')

In [ ]:
from gravity_obs import ob_add_target
def dual_off_for_target(grav, target, folder_name=None,
                         usename=False, verify=False):
    ao_mode = "LGS_VIS"
    tel_alpha = target['ra_2000_hms']
    tel_delta = target['dec_2000_dms']

    if usename:
        obname = f"{ao_mode}_{target['name']}".replace(' ', '_')
    else:
        obname = f"{ao_mode}_{tel_alpha.split('.')[0].replace(':', '')}{tel_delta.split('.')[0].replace(':', '')}_{target['Grp']:4.1f}_{target['K']:4.1f}".replace(' ', '_')


    ob, obV = grav.create_OB(obname, folder_name=folder_name, overwrite=True)
    obsDes = ob['obsDescription']
    obsDes['name'] = target['name']
    obsDes['userComments'] = 'Dual on-axis observation test'

    ob_add_target(ob, target['name'], 
                    ra = target['ra_2000_hms'], 
                    dec = target['dec_2000_dms'],
                    pma = target['pmra']/1e3,
                    pmd = target['pmdec']/1e3)
    ob, obV = grav.save_OB(obname, folder_name=folder_name)


    dit, ndit = 3, 40
    grav.add_GRAVITY_dual_offaxis_acq(name=obname, folder_name=folder_name,
                                    sc_name=target['name'],
                                    sc_kmag=target['K'],
                                    sc_hmag=target['H'],
                                    sobj_x = target['sobj_x'],
                                    sobj_y = target['sobj_y'],
                                    sc_plx=target['parallax']/1e3,
                                    ft_kmag=target['K_ft'],
                                    ft_hmag=target['H_ft'],
                                    ft_plx=target['parallax_ft']/1e3,
                                    ngs_mag=target['Grp_ft'],
                                    ao_mode=ao_mode,
                                    baseline=['UTs'])
    grav.add_GRAVITY_dual_obs_exp(obname, folder_name=folder_name,
                                    dit=dit, ndit_obj=ndit, ndit_sky=ndit,)

    # optional verification
    if verify:
        obID = ob['obId']
        response, _ = grav.api.verifyOB(obID,True)
        if response['observable']:
            print('*** Congratulations. Your OB' , obID, ob['name'], 'is observable!')
        else:
            print('OB', obID, 'is >>not observable<<. See messages below.')
            print(' ', '\n  '.join(response['messages']))

In [7]:
grav = p2t.p2api_GRAVITY('116.29H7.001', 'GPLUSTeam', 'GPAOwFRINGES', no_warning=True)
root_folder = 'TTR-116.0046 October First Light'
grav.set_rootFolder(root_folder)

In [8]:
folder_name = 'dual_offaxis_ob_test'
try:
    grav.get_folder(folder_name)
except Exception:
    grav.create_folder(folder_name, overwrite=True)

In [9]:
target_names = ["Gaia DR3 2341844906852746368", "Gaia DR3 2339430821699529728"]
for obj_name in target_names:
    target = tb.loc[obj_name]
    dual_off_for_target(grav, target, folder_name=folder_name)

KeyError: 'K'